# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate as tabulate
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from matplotlib.colors import TwoSlopeNorm

# Importing Electricity Data

Simplifying electricity data to only look at two regions: Mid-Atlantic (NJ, NY, PA) and West North Central (ND, SD, MN, MO, KS, IA).

In [ ]:
elecdf=pd.read_csv(r"C:\Users\tonya\OneDrive - Rutgers University\Documents\Rutgers\Research\NRELAnalysis\Electricity.csv")

elecMAWNC=elecdf[elecdf['Division'].isin(['Mid Atlantic', 'West North Central'])]
elecMAWNC=elecMAWNC.drop('Used on Plant (Cent/s)', axis=1)
elecMAWNC=elecMAWNC.drop('Total Electriciy (Cent/s)', axis=1)
elecMAWNC['Price ($/kWh)']=elecMAWNC['Industrial 2024 (Cent/kWh)']/100

# Chemical Pricing

Pricing of Biomass, Ethanol, CO2 and Formic Acid. CO2 is assuming 45Q credit of `$12/ton`. Formic Acid is assuming electrolytic conversion of CO2 from generated electricity on plant. 

In [ ]:
chemprice={
    'Compound': ['Biomass', 'Ethanol', 'CO2', 'Formic Acid'], 
    'Price': [46.8, 720, 12, 550], 
    'Mass Flow (kg/h)': [104167, 21673, 91746, 7848], 
}
chemprice_df=pd.DataFrame(chemprice)
chemprice_df['Mass Flow (ton/h)']=chemprice_df['Mass Flow (kg/h)']/1000
chemprice_df['Mass Flow (kg/s)']=chemprice_df['Mass Flow (kg/h)']/3600
chemprice_df['Cost ($/h)']=chemprice_df['Price']*chemprice_df['Mass Flow (ton/h)']
chemprice_df.at[3, 'Cost ($/h)']=chemprice_df.at[3, 'Price']*chemprice_df.at[3, 'Mass Flow (ton/h)']/0.85
chemprice_df['Cost ($/s)']=chemprice_df['Cost ($/h)']/3600
chemprice_df['Cost (c/s)']=chemprice_df['Cost ($/s)']*100

# Base NREL Plant

Will assume the plant can utilize the 45Q Carbon Credit at $12/ton of CO2.

In [ ]:
basecase_df=chemprice_df.loc[[0, 1, 2], ['Compound', 'Cost (c/s)', 'Mass Flow (kg/s)']]
basecase_dict=pd.Series(basecase_df['Cost (c/s)'].values, index=basecase_df['Compound']).to_dict()
basekgCO2=basecase_df.loc[basecase_df['Compound']=='CO2', 'Mass Flow (kg/s)'].values[0]
profits1, profits2 =[], []
conv=(60*60*8000)/100
for index, row in elecMAWNC.iterrows():
    elec_price=row['Sold to Grid (Cent/s)']
    cost_bio, cost_eth, cost_CO2=basecase_dict['Biomass'], basecase_dict['Ethanol'], basecase_dict['CO2']
    profit1=(cost_eth+elec_price-cost_bio-cost_CO2)*conv
    profits1.append({'State': row['State'], 'Revenue with CO2 tax ($/yr)': profit1})
    profit2=(cost_eth+elec_price-cost_bio)*conv
    profits2.append({'State': row['State'], 'Revenue with no CO2 tax ($/yr)': profit2})

df_profits1, df_profits2=pd.DataFrame(profits1), pd.DataFrame(profits2)
df_profitmerg=pd.merge(df_profits1, df_profits2, on='State', how='outer')
df_profits_sort=df_profitmerg.sort_values(by='Revenue with CO2 tax ($/yr)', ascending=False)

max_prof1=df_profits1['Revenue with CO2 tax ($/yr)'].max()
max_prof2=df_profits2['Revenue with no CO2 tax ($/yr)'].max()
min_prof1=df_profits1['Revenue with CO2 tax ($/yr)'].min()
min_prof2=df_profits2['Revenue with no CO2 tax ($/yr)'].min()
print(f"Maximum revenue with CO2 tax: {max_prof1:,.0f} $/yr")
print(f"Maximum revenue with no CO2 tax: {max_prof2:,.0f} $/yr")
print(f"Minimum revenue with CO2 tax: {min_prof1:,.0f} $/yr")
print(f"Minimum revenue with no CO2 tax: {min_prof2:,.0f} $/yr")

mill=1000000
bar_w=0.4
x=np.arange(len(df_profits_sort))

fig, (ax1, ax2) =plt.subplots(1, 2, figsize=(15, 6))
ax1.bar(x-bar_w/2, df_profits_sort['Revenue with CO2 tax ($/yr)']/mill, width=bar_w, label='Revenue with CO2 Tax ($/yr)', color='blue')
ax1.bar(x+bar_w/2, df_profits_sort['Revenue with no CO2 tax ($/yr)']/mill, width=bar_w, label='Revenue with no CO2 Tax ($/yr)', color='orange')
ax1.set_xlabel("State")
ax1.set_ylabel("Annual Revenue (Million $/yr)")
ax1.set_title("Revenue Comparison: With or Without CO2 Tax at $12/ton")
ax1.set_xticks(x)
ax1.set_ylim(80, 100)
ax1.set_xticklabels(df_profits_sort['State'], rotation=45)
ax1.legend(loc="best", bbox_to_anchor=(1,1))

NRELCap, NRELOp=152900000, 65900000
df_profits_sort[['Yearly Profit with CO2 Tax', 'Yearly Profit no CO2 Tax']]=df_profits_sort[['Revenue with CO2 tax ($/yr)', 'Revenue with no CO2 tax ($/yr)']]-NRELOp
df_profits_sort[['Payback Period with CO2 Tax', 'Payback Period no CO2 Tax']]=NRELCap/df_profits_sort[['Yearly Profit with CO2 Tax', 'Yearly Profit no CO2 Tax']]
df_profits_sort['Payback Period with CO2 Tax']=df_profits_sort['Payback Period with CO2 Tax'].round(2)
df_profits_sort['Payback Period no CO2 Tax']=df_profits_sort['Payback Period no CO2 Tax'].round(2)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.colheader_justify', 'center')
# df_profits_sort.to_excel("profits_report.xlsx",index=False)

ax2.bar(x-bar_w/2, df_profits_sort['Payback Period with CO2 Tax'], width=bar_w, label='Payback Period with CO2 Tax', color='blue')
ax2.bar(x+bar_w/2, df_profits_sort['Payback Period no CO2 Tax'], width=bar_w, label='Payback Period no CO2 Tax', color='orange')
ax2.set_xlabel("State")
ax2.set_ylabel("Payback Periods (years)")
ax2.set_title("Payback Period Comparison: With or Without CO2 Tax at $12/ton")
ax2.set_xticks(x)
ax2.set_ylim(0, max(df_profits_sort[['Payback Period with CO2 Tax', 'Payback Period no CO2 Tax']].max()*1.1))
ax2.set_xticklabels(df_profits_sort['State'], rotation=45)
ax2.legend(loc="best", bbox_to_anchor=(1,1))
plt.tight_layout
plt.show()

# Co-Production of Formic Acid and Ethanol

Looking at utilizing 12.8MW of electricity generated from NREL plant and 90 tons of CO2 produced to generate Formic Acid.

In [ ]:
FAthermo={
    'Compound': ['CO2', 'H2O', 'O2', 'CH2O2'], 
    'H form': [-393.5, -285.8, 0, -425], 
    'G form': [-394.4, -237.1, 0, -361.4], 
}
FAtherm_df=pd.DataFrame(FAthermo)
FA=FAtherm_df.loc[FAtherm_df['Compound']=='CH2O2']
CO2=FAtherm_df.loc[FAtherm_df['Compound']=='CO2']
H2O=FAtherm_df.loc[FAtherm_df['Compound']=='H2O']
H_rxn=FA['H form'].values[0] - (CO2['H form'].values[0] + H2O['H form'].values[0])
G_rxn=FA['G form'].values[0] - (CO2['G form'].values[0] + H2O['G form'].values[0])
print(f"H rxn: {H_rxn:.1f}")
print(f"G rxn: {G_rxn:.1f}")

genelec=12797
FAprod=genelec/G_rxn
print(f"Formic Acid Produced: {FAprod:.1f} mol/s")

PlantCO2= {
    'Area': ['Combustor', 'Fermenter'], 
    'Mass Flow (kg/h)': [73396, 18350], 
}
plantco2_df=pd.DataFrame(PlantCO2)
plantco2_df['Mass Flow (kg/s)']=(plantco2_df['Mass Flow (kg/h)']/3600)
plantco2_df['Form Mass Flow (kg/s)']=plantco2_df['Mass Flow (kg/s)'].apply(lambda x: f"{x:.2f}")
plantco2_df['Mole Flow (mol/s)']=(plantco2_df['Mass Flow (kg/s)']/44.01)*1000
print(plantco2_df[['Area', 'Mass Flow (kg/h)', 'Form Mass Flow (kg/s)', 'Mole Flow (mol/s)']].to_string(index=False, justify='center'))

co2conv={
    'Area': ['Combustor', 'Fermenter', 'Overall'], 
    'CO2 Mole Flow': [463, 116, 463+116],
}
co2conv_df=pd.DataFrame(co2conv)
co2conv_df['CO2 Conversion (%)']=(FAprod/co2conv_df['CO2 Mole Flow'])*100
print(co2conv_df.to_string(index=False, justify='center'))

remco2=((co2conv_df.loc[co2conv_df['Area']=='Overall', 'CO2 Mole Flow'])-FAprod).values[0]*44.01/1000
print(f"Remaining CO2: {remco2:.2f} kg/s")

FAmass=(FAprod*46.03)/1000 #kg/s
priceFA=((chemprice_df.loc[chemprice_df['Compound']=='Formic Acid', 'Price'].values[0])/10)*FAmass/0.85
print(f"Mass of FA Produced: {FAmass:.2f} kg/s")
print(f"Profit of FA Produced: {priceFA:.2f} cent/s")

maxprofitnt=priceFA+(chemprice_df.loc[chemprice_df['Compound']=='Ethanol', 'Cost (c/s)'].values[0])-(chemprice_df.loc[chemprice_df['Compound']=='Biomass', 'Cost (c/s)'].values[0])
maxprofitwt=maxprofitnt-(((chemprice_df.loc[chemprice_df['Compound']=='CO2', 'Price'].values[0])/10)*remco2)
print(f"Max Product Profit of Co-Production No CO2 Tax: ${maxprofitnt*(3600*8000/100):,.0f}/yr")
print(f"Max Product Profit of Co-Production With CO2 Tax: ${maxprofitwt*(3600*8000/100):,.0f}/yr")

In [ ]:
minrevnt=df_profitmerg['Revenue with no CO2 tax ($/yr)'].min()
maxrevnt=df_profitmerg['Revenue with no CO2 tax ($/yr)'].max()
minrevwt=df_profitmerg['Revenue with CO2 tax ($/yr)'].min()
maxrevwt=df_profitmerg['Revenue with CO2 tax ($/yr)'].max()

maxprofwt=maxprofitwt*3600*8000/100
maxprofnt=maxprofitnt*3600*8000/100
maxincwt=maxprofwt-maxrevwt
increase={
    'Profit Type': ['With CO2 Tax', 'No CO2 Tax'], 
    'Maximum Increase': [maxprofwt-minrevwt, maxprofnt-minrevnt], 
    'Minimum Increase': [maxprofwt-maxrevwt, maxprofnt-maxrevnt],
}
inc_df=pd.DataFrame(increase)
print(inc_df.to_string(index=False, justify='center'))

perincrease={
    'Profit Type': ['With CO2 Tax', 'No CO2 Tax'], 
    'Maximum Increase': [((maxprofwt-minrevwt)/minrevwt)*100, ((maxprofnt-minrevnt)/minrevnt)*100], 
    'Minimum Increase': [((maxprofwt-maxrevwt)/maxrevwt)*100, ((maxprofnt-maxrevnt)/maxrevnt)*100],
}
perinc_df=pd.DataFrame(perincrease)
print(perinc_df.to_string(index=False, justify='center'))

# Full Production of Formic Acid from CO2

Lets assume we disregard ethanol entirely. Instead, lets fully convert biomass into CO2 and then fully into Formic Acid. We will assume the 12.8MW of electricity generated is still fully used and will calculate the remaining electricity required.

In [ ]:
stoy=60*60*8000
massflowbio=28.9 #kg/s
mwbio=162.52 #kg/kmol
pricebio=(massflowbio/1000)*46.8*stoy #$/yr
molebio=massflowbio/mwbio #kmol/s
moleflowco2=molebio*6 #can produce 6 moles of CO2 from one mole of biomass
print(f"Biomass Cost: ${pricebio:,.0f}/yr")

FAprodkmol=FAprod/1000 #kmol/s
flowco2rem=moleflowco2-FAprodkmol #kmol/s
reqelectricity=(flowco2rem*1000)*G_rxn #kW
remFA=flowco2rem*46.03 #kg/s

totFA=(remFA+FAmass)/1000 #ton/s
totFAprice=(550*totFA*stoy)/0.85 #$/yr
print(f"Formic Acid Sales: ${totFAprice:,.0f}/yr")

elecMAWNC_FA=elecMAWNC.drop('Sold to Grid (Cent/s)', axis=1)
elecMAWNC_FA['Electricity Cost ($/yr)']=(elecMAWNC_FA['Industrial 2024 (Cent/kWh)']*reqelectricity*stoy)/(100*3600)
elecMAWNC_FA['Plant Profit ($/yr)']=totFAprice-elecMAWNC_FA['Electricity Cost ($/yr)']-pricebio
print(elecMAWNC_FA.to_string(index=False, justify='center'))

While full production of Formic Acid is the most profitable, this is 150% of the global Formic Acid market. Therefore, will not move forward with this.

# Electrolyzer Capital Costs

Since we know the product profitability of co-production, lets examine the capital costs associated with an electrolyzer so we can do a more complete cost analysis. 

To start, lets assume we have a stand-alone electrolyzer that produces the co-production amount of Formic Acid. We can also assume that CO2 will not be free in this case.

In [ ]:
refEV=1.75 #Reference electrolyzer voltage in V
refCD=175 #Reference electrolyzer current density in mA/cm^2
FApd=FAmass*86400 #kg/s * 60 s/min * 60 min/h * 24 h/day
print(f"Mass of FA Produced: {FApd:,.0f} kg/day")

#Electrolyzer Assumptions
cellvolt=3.5 #in V
prodsel=0.94 #85% product selectiviy, assuming this is where purity of product comes into play. Also is the Faradaic efficiency
elpm=2 #2 electrons per mole of Formic Acid
MWFA=46.03 #molecular weight of FA in g/mol
fc=96480 #Faradays Constant in C/mol
kgtog=1000 #conversion of kg to g
curd=3 #current density in A/cm^2
totcur=((FAmass*kgtog)/MWFA)*elpm*fc/prodsel
elecareacalc=totcur/curd/(10**4)
print(f"Electrolyzer Area: {elecareacalc:.1f} m^2")
powerneed=(totcur*cellvolt)/(10**6)
print(f"Total Power Required: {powerneed:.1f} MW")
# realpow=powerneed-(genelec/1000)
realpow=powerneed
print(f"Power required after utilizing generated power: {realpow:.1f} MW")
elecreq=0.12*realpow*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")

In [ ]:
ezrcost={
    'Base Cost ($/kW)': [250, 550, 2640], 
}
ezrcost_df=pd.DataFrame(ezrcost)
ezrcost_df['Installed Cost ($/m^2)']=ezrcost_df['Base Cost ($/kW)']*cellvolt*curd/1000*(10**4)*1.2
ezrcost_df['Electrolyzer Capital Cost ($)']=ezrcost_df['Installed Cost ($/m^2)']*elecareacalc

annFA=FApd*(8000/24)/1000 #kg/day * 1day/24h* 8000h/yr *1ton/1000 kg
sellannFA=annFA*550
print(f"Annual Profit of Formic Acid: ${sellannFA:,.0f}/yr")
annCO2=annFA*(44.01/46.03) #tonFA/yr*1000kg/1ton*1kmolFA/46.03kg*1kmolCO2/1kmolFA*44.01kg/1kmolCO2*1ton/1000kg
sellannCO2=annCO2*40
print(f"Annual Price of CO2 at $40/ton: ${sellannCO2:,.0f}/yr")
yearprof=sellannFA-elecreq-sellannCO2
print(f"Annual Profit of Electrolyzer: ${yearprof:,.0f}/yr")

ezrcost_df['Payback Period (years)']=ezrcost_df['Electrolyzer Capital Cost ($)']/yearprof
print(ezrcost_df.to_string(index=False, justify='center'))

A standalone electrolyzer in New Jersey producing 188 tons of Formic Acid per day at `$550/ton` can pay back its capital cost between 0.15 and 1.58 years, depending on the base cost of the electrolyzer. This also assumes CO2 is bought directly at a price of `$12/ton`. Biomass was not converted to CO2 in this scenario.

We have a range of capital costs for an electrolyzer of CO2 to Formic Acid, lets see how the payback period of the NREL plant compares to the co-production plant.

In [ ]:
print(f"Max Product Profit of Co-Production No CO2 Tax: ${maxprofitnt*(3600*8000/100):,.0f}/yr")
print(f"Max Product Profit of Co-Production With CO2 Tax: ${maxprofitwt*(3600*8000/100):,.0f}/yr")
elecreq=0.1198*realpow*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")
NRELCap, NRELOp=152900000, 65900000


capcost_df=ezrcost_df.drop(columns=['Installed Cost ($/m^2)', 'Payback Period (years)'])
capcost_df['Co-Production Capital Cost ($)']=capcost_df['Electrolyzer Capital Cost ($)']+NRELCap
capcost_df['With CO2 Tax Payback Period (years)']=capcost_df['Co-Production Capital Cost ($)']/((maxprofitwt*(3600*8000/100))-elecreq-NRELOp)
capcost_df['No CO2 Tax Payback Period (years)']=capcost_df['Co-Production Capital Cost ($)']/((maxprofitnt*(3600*8000/100))-elecreq-NRELOp)
print(capcost_df.to_string(index=False))

In [ ]:
elecMAWNC_FANREL=elecMAWNC_FA.drop(columns=['Electricity Cost ($/yr)', 'Plant Profit ($/yr)'])
elecMAWNC_FANREL['Industrial 2024 ($/kWh)']=elecMAWNC_FANREL['Industrial 2024 (Cent/kWh)']/100
elecMAWNC_FANREL=elecMAWNC_FANREL.drop('Industrial 2024 (Cent/kWh)', axis=1)
elecMAWNC_FANREL['Electricity Required ($/yr)']=elecMAWNC_FANREL['Industrial 2024 ($/kWh)']*realpow*1000*8000

base_elcost=[250, 550, 2640]
capital_costs={cost: capcost_df.loc[capcost_df['Base Cost ($/kW)']==cost, 'Co-Production Capital Cost ($)'].values[0]
               for cost in base_elcost}

for cost in base_elcost:
    coprodcap=capital_costs[cost]
    elecMAWNC_FANREL[f'${cost} Plant Profit with Tax ($/yr)']=(maxprofitwt*(3600*8000/100))-elecMAWNC_FANREL['Electricity Required ($/yr)']-NRELOp
    elecMAWNC_FANREL[f'${cost} Plant Profit No Tax ($/yr)']=(maxprofitnt*(3600*8000/100))-elecMAWNC_FANREL['Electricity Required ($/yr)']-NRELOp
    elecMAWNC_FANREL[f'${cost} Payback Period with Tax (years)']=coprodcap/elecMAWNC_FANREL[f'${cost} Plant Profit with Tax ($/yr)']
    elecMAWNC_FANREL[f'${cost} Payback Period No Tax (years)']=coprodcap/elecMAWNC_FANREL[f'${cost} Plant Profit No Tax ($/yr)']

In [ ]:
bc_dataframes={}

for cost in base_elcost:
    bc_dataframes[cost]=elecMAWNC_FANREL[['State', f'${cost} Plant Profit with Tax ($/yr)', f'${cost} Plant Profit No Tax ($/yr)']].copy()
    bc_dataframes[cost].loc[:, f'${cost} Payback Period with Tax (years)']=capital_costs[cost]/bc_dataframes[cost][f'${cost} Plant Profit with Tax ($/yr)']
    bc_dataframes[cost].loc[:, f'${cost} Payback Period No Tax (years)']=capital_costs[cost]/bc_dataframes[cost][f'${cost} Plant Profit No Tax ($/yr)']
plantprofmerg=bc_dataframes[250].merge(bc_dataframes[550], on='State', how='outer').merge(bc_dataframes[2640], on='State', how='outer')

bc250maxwt=bc_dataframes[250]['$250 Plant Profit with Tax ($/yr)'].max()
bc250maxnt=bc_dataframes[250]['$250 Plant Profit No Tax ($/yr)'].max()

bc250minwt=bc_dataframes[250]['$250 Plant Profit with Tax ($/yr)'].min()
bc250minnt=bc_dataframes[250]['$250 Plant Profit No Tax ($/yr)'].min()

In [ ]:
maxprof1per=(max_prof1-bc250maxwt)/max_prof1*100
maxprof2per=(max_prof2-bc250maxnt)/max_prof2*100
minprof1per=(min_prof1-bc250minwt)/min_prof1*100
minprof2per=(min_prof2-bc250minnt)/min_prof2*100

proftab={
    'Status': ['With Tax', 'No Tax'], 
    'Base Case Max Profit': [max_prof1, max_prof2], 
    'Base Case Min Profit': [min_prof1, min_prof2],
    'Electrolyzer Max Profit': [bc250maxwt, bc250maxnt], 
    'Electrolyzer Min Profit': [bc250minwt, bc250minnt], 
    'Max Percent': [100-maxprof1per, 100-maxprof2per], 
    'Min Percent': [100-minprof1per, 100-minprof2per],
}

proftab=pd.DataFrame(proftab)
print(proftab.to_string(index=False, justify='center'))

In [ ]:
states=bc_dataframes[250]['State'].tolist()
x=np.arange(len(states))
width=0.4
fig, ax5=plt.subplots(1,1)
bars1=ax5.bar(x-width/2, bc_dataframes[250]['$250 Plant Profit with Tax ($/yr)']/mill, width, label='Profit with Tax')
bars2=ax5.bar(x+width/2, bc_dataframes[250]['$250 Plant Profit No Tax ($/yr)']/mill, width, label='Profit No Tax')
ax5.set_xlabel("State")
ax5.set_ylabel("Plant Profit ($Million/yr)")
ax5.set_title("Plant Profits")
ax5.set_xticks(x)
ax5.set_xticklabels(states, rotation=45)
ax5.legend(loc="best", bbox_to_anchor=(1,1))

In [ ]:
payback_col={
    250: ['$250 Payback Period with Tax (years)', '$250 Payback Period No Tax (years)'], 
    550: ['$550 Payback Period with Tax (years)', '$550 Payback Period No Tax (years)'], 
    2640: ['$2640 Payback Period with Tax (years)', '$2640 Payback Period No Tax (years)']
}
states=bc_dataframes[250]['State'].tolist()
x=np.arange(len(states))
width=0.15
fig, ax6=plt.subplots(1,1)
offsets=[-2, -1, 0, 1, 2, 3]
for i, (cost, cols) in enumerate(payback_col.items()):
    ax6.bar(x+offsets[2*i]*width, bc_dataframes[cost][cols[0]], width, label=f'{cost} Payback with Tax')
    ax6.bar(x+offsets[2*i+1]*width, bc_dataframes[cost][cols[1]], width, label=f'{cost} Payback No Tax')
ax6.set_xlabel("State")
ax6.set_ylabel("Payback Period (years)")
ax6.set_title("Payback Period Comp")
ax6.set_xticks(x)
ax6.set_xticklabels(states, rotation=45)
ax6.legend(loc="best", bbox_to_anchor=(1,1))
plt.show()

# Formic Acid Production

Looking at a separate facility to produce Formic Acid from CO2 by varying the input flowrate of CO2 and determining its impact on Formic Acid production, Electricity required and the price of that amount of CO2. Will then determine the annual product profit of the plant.

Starting with a plant in New Jersey where CO2 costs `$100/ton`. 

In [ ]:
FAchem={
    'Compound': ['Biomass', 'Formic Acid', 'CO2'], 
    'Price ($/ton)': [46.8, 550, 100], 
}
FA_df=pd.DataFrame(FAchem)
CO2flow=list(range(5, 101, 5))
GEFA=270.1
yearh=8000
MWCO2=44.01
tong=(1e-6)
NJprice=0.1198
FAprice=FA_df.loc[FA_df['Compound']=='Formic Acid', 'Price ($/ton)'].values[0]
CO2price=FA_df.loc[FA_df['Compound']=='CO2', 'Price ($/ton)'].values[0]

df_CO2flow=pd.DataFrame({'CO2 Flow Rate (mol/s)': CO2flow})
df_CO2flow['FA Selling Price']=df_CO2flow['CO2 Flow Rate (mol/s)']*FAprice*MWFA*tong
df_CO2flow['Cost of Electric']=df_CO2flow['CO2 Flow Rate (mol/s)']*NJprice*GEFA/3600
df_CO2flow['Cost of CO2']=df_CO2flow['CO2 Flow Rate (mol/s)']*CO2price*MWCO2*tong
df_CO2flow['Annual Product Profit C']=(df_CO2flow['FA Selling Price']-df_CO2flow['Cost of Electric']-df_CO2flow['Cost of CO2'])*60*60*yearh

df_CO2flow=df_CO2flow.round({'FA Selling Price':2, 'Cost of Electric': 2, 'Cost of CO2': 2})
df_CO2flow['Annual Product Profit']=df_CO2flow['Annual Product Profit C'].map('{:,.0f}'.format)
print(df_CO2flow.drop(['Annual Product Profit C'], axis=1).to_string(index=False, justify='center'))

In [ ]:
BCtoICconv=refEV*refCD/1000/1000*(10**4)*1.2

newFAconv=df_CO2flow.copy()
newFAconv['FA Mass Flow (kg/s) C']=newFAconv['CO2 Flow Rate (mol/s)']*MWFA/1000
newFAconv['FA Mass Flow (kg/day) C']=newFAconv['FA Mass Flow (kg/s) C']*60*60*24
newFAconv['FA Mass Flow (kg/s)']=newFAconv['FA Mass Flow (kg/s) C'].apply(lambda x: f"{x:.2f}")
newFAconv['FA Mass Flow (kg/day)']=newFAconv['FA Mass Flow (kg/day) C'].apply(lambda x: f"{x:,.0f}")
newFAconv['Total Current (A)']=((newFAconv['FA Mass Flow (kg/s) C']*kgtog)/MWFA)*elpm*fc/prodsel
newFAconv['Electrolyzer Area (m^2)']=newFAconv['Total Current (A)']/curd/(10**4)
newFAconv['Power Needed (MW)']=(newFAconv['Total Current (A)']*cellvolt)/(10**6)
newFAconv['BC $250 Cap Cost']=250*BCtoICconv*newFAconv['Electrolyzer Area (m^2)']
newFAconv['BC $550 Cap Cost']=550*BCtoICconv*newFAconv['Electrolyzer Area (m^2)']
newFAconv['BC $2640 Cap Cost']=2640*BCtoICconv*newFAconv['Electrolyzer Area (m^2)']
print(newFAconv[['CO2 Flow Rate (mol/s)', 'FA Mass Flow (kg/day)', 'Electrolyzer Area (m^2)']])
print(newFAconv[['BC $250 Cap Cost', 'BC $550 Cap Cost', 'BC $2640 Cap Cost']])

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(newFAconv['CO2 Flow Rate (mol/s)'], newFAconv['BC $250 Cap Cost']/mill, marker='o', label="Base Cost $250")
plt.plot(newFAconv['CO2 Flow Rate (mol/s)'], newFAconv['BC $550 Cap Cost']/mill, marker='s', label="Base Cost $550")
plt.plot(newFAconv['CO2 Flow Rate (mol/s)'], newFAconv['BC $2640 Cap Cost']/mill, marker='^', label="Base Cost $2640")
plt.xlabel("CO2 Flow Rate (mol/s)")
plt.ylabel("Electrolyzer Capital Cost (Million $)")
plt.title("CO2 Flow Rate vs Electrolyzer Capital Cost")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
co2_price_range=np.linspace(-100, 100, 9).round(0)
electricity_price_range=dict(zip(elecMAWNC['State'], elecMAWNC['Price ($/kWh)']))
df_CO2flow[['FA Selling Price', 'Cost of Electric', 'Cost of CO2']]=df_CO2flow[['FA Selling Price', 'Cost of Electric', 'Cost of CO2']].astype(float)
results=[]

for co2_price in co2_price_range:
    for state, electricity_price in electricity_price_range.items():
        profit=(df_CO2flow['FA Selling Price']-
                (df_CO2flow['CO2 Flow Rate (mol/s)']*electricity_price*GEFA/3600)-
                (df_CO2flow['CO2 Flow Rate (mol/s)']*co2_price*MWCO2*tong)
               )*60*60*yearh/mill
        for index, p in enumerate(profit):
            results.append({
                'CO2 Flow Rate (mol/s)': df_CO2flow['CO2 Flow Rate (mol/s)'][index],
                'CO2 Price': co2_price, 
                'Electricity Price C': electricity_price,
                'State': state,
                'Profit ($ Million)': p
            })

df_results=pd.DataFrame(results)
df_results["Electricity Price"]=df_results["Electricity Price C"].map(lambda x: f"{x:.2f}")
columns=['CO2 Flow Rate (mol/s)', 'CO2 Price', 'Electricity Price', 'State', 'Profit ($ Million)']
df_results=df_results[columns]
print(df_results)
df_filt=df_results.loc[(df_results["CO2 Flow Rate (mol/s)"]==100) & (df_results["CO2 Price"]==100)]
print(df_filt)

In [ ]:
ppelecres=df_results.merge(
    newFAconv[['CO2 Flow Rate (mol/s)', 'BC $250 Cap Cost', 'BC $550 Cap Cost', 'BC $2640 Cap Cost']], 
    on='CO2 Flow Rate (mol/s)',
    how='left'
)
prof_dol=ppelecres['Profit ($ Million)']*1e6
for cost in ['250', '550', '2640']:
    ppelecres[f'Payback ${cost} BC']=ppelecres[f'BC ${cost} Cap Cost']/prof_dol

ppfilt=ppelecres.query("State=='New Jersey'")
ppelecres['Profit ($ Million)'].agg(['min', 'max'])
# print(ppfilt)

## Heatmap of CO2 Price, Electricity Price and Annual Profit of Plant

In [ ]:
df_pivot=df_results.pivot_table(index='CO2 Price', columns='Electricity Price', values='Profit ($ Million)')
df_pivot=df_pivot.sort_index().sort_index(axis=1)
df_pivot.index=df_pivot.index.astype(float)
df_pivot.columns=df_pivot.columns.astype(float)
fig, ax = plt.subplots()
sns.heatmap(df_pivot, annot=False, cmap='coolwarm', cbar_kws={'label': 'Annual Profit ($ Million)'}, ax=ax)
yticklabels=ax.get_yticklabels()
yticklabels=[label.get_text() for label in yticklabels]
yticklabels=[str(int(float(label))) for label in yticklabels]
ax.set_yticklabels(yticklabels)
plt.show()

## 3D Scatter Plot of CO2 Flow Rate/Price and Profit

In [ ]:
fig=plt.figure(figsize=(10, 8))
ax=fig.add_subplot(111, projection="3d")

X=df_results["CO2 Flow Rate (mol/s)"]
Y=df_results["CO2 Price"]
Z=df_results["Profit ($ Million)"]

ax.scatter(X, Y, Z, c=Z, cmap='coolwarm', marker='o')
ax.set_xlabel("CO2 Flow Rate (mol/s)")
ax.set_ylabel("CO2 Price")
ax.set_zlabel("Profit ($)", fontsize=10, labelpad=5)
ax.set_title("3D Vis of Profit vs CO2 Flow Rate and Price")
ax.set_zlim(min(Z), max(Z))
ax.view_init(elev=25, azim=125)
plt.show()

# plt.figure(figsize=(10, 7))
# for flow_rate in df_results["CO2 Flow Rate (mol/s)"].unique():
#     subset=df_results[df_results["CO2 Flow Rate (mol/s)"]==flow_rate]
#     plt.plot(subset["CO2 Price"], subset["Profit ($ Million)"], label=f"Flow Rate = {flow_rate} mol/s")
# plt.xlabel("CO2 Price ($/ton)")
# plt.ylabel("Profit ($)")
# plt.title("Impact of CO2 Price on Profit for Different CO2 Flow Rates")
# plt.legend(loc="upper left", bbox_to_anchor=(1,1))
# plt.grid(True)
# plt.show()

## Heatmap of CO2 Price, Formic Acid Price and Plant Profit

In [ ]:
FA_price_range=np.arange(0, 1000, 100)

df_CO2flow[['FA Selling Price', 'Cost of Electric', 'Cost of CO2']]=df_CO2flow[['FA Selling Price', 'Cost of Electric', 'Cost of CO2']].astype(float)
resultsFA=[]
co2_flow=df_CO2flow['CO2 Flow Rate (mol/s)']
FA_factor=MWFA*tong
elec_factor=GEFA/3600
co2_factor=MWCO2*tong

for co2_price in co2_price_range:
    for state, electricity_price in electricity_price_range.items():
        for FA_price in FA_price_range:
            profit=((co2_flow*FA_price*FA_factor)-
                (co2_flow*electricity_price*elec_factor)-
                (co2_flow*co2_price*co2_factor)
                   )*60*60*yearh/mill
            for index, p in enumerate(profit):
                resultsFA.append({
                    'CO2 Flow Rate (mol/s)': co2_flow[index],
                    'CO2 Price': co2_price, 
                    'Electricity Price C': electricity_price,
                    'Formic Acid Price C': FA_price,
                    'State': state,
                    'Profit ($ Million)': p
            })
df_resultsFA=pd.DataFrame(resultsFA)
df_resultsFA["Electricity Price"]=df_resultsFA["Electricity Price C"].map(lambda x: f"{x:.2f}")
df_resultsFA["Formic Acid Price"]=df_resultsFA["Formic Acid Price C"].map(lambda x: f"{x:.0f}")

columns=['CO2 Flow Rate (mol/s)', 'CO2 Price', 'Formic Acid Price', 'Electricity Price', 'State', 'Profit ($ Million)']
df_resultsFA=df_resultsFA[columns]

pivot_FA=df_resultsFA.pivot_table(
    index='CO2 Price',
    columns='Formic Acid Price', 
    values='Profit ($ Million)',
    aggfunc='mean'
)

plt.figure(figsize=(8, 5))
norm=TwoSlopeNorm(vmin=pivot_FA.min().min(), vcenter=0, vmax=pivot_FA.max().max())
sns.heatmap(pivot_FA, cmap='coolwarm', annot=True, cbar_kws={'label': 'Profit ($ Million)'})

# for state in df_resultsFA['State'].unique():
#     state_data=df_resultsFA[df_resultsFA['State']==state]
#     state_line_data=state_data.groupby(['CO2 Price', 'Formic Acid Price'])['Profit ($ Million)'].mean().reset_index()
#     plt.plot(state_line_data['Formic Acid Price'], state_line_data['Profit ($ Million)'], label=state, marker='o')

# plt.legend(title="State", bbox_to_anchor=(1,1), loc='best')
plt.tight_layout()
plt.show()

In [ ]:
%%time
# Define your specific price ranges for the new dataset
fa_price_range = np.arange(0, 201, 1)  # Formic Acid Price range from 0 to 200
co2_price_range = np.arange(-100, 101, 1)  # CO2 Price range from -100 to 100
electricity_price_range=dict(zip(elecMAWNC['State'], elecMAWNC['Price ($/kWh)']))  # Electricity Price range
state_div=dict(zip(elecMAWNC['State'], elecMAWNC['Division']))

# Initialize an empty list to store the results
resultsFA_new = []

# Loop through the new ranges and calculate profit
for co2_price in co2_price_range:
    for state, electricity_price in electricity_price_range.items():  # Use the state and its associated electricity price
        division=state_div[state]
        for FA_price in fa_price_range:
            # Calculate profit
            profit = ((co2_flow * FA_price * FA_factor) -
                      (co2_flow * electricity_price * elec_factor) -
                      (co2_flow * co2_price * co2_factor)
                     ) * 60 * 60 * yearh
            # Store results for each combination
            for index, p in enumerate(profit):
                resultsFA_new.append({
                    'CO2 Flow Rate (mol/s)': co2_flow[index],
                    'CO2 Price': co2_price, 
                    'Electricity Price C': electricity_price,
                    'Formic Acid Price C': FA_price,
                    'State': state,
                    'Division': division,
                    'Profit ($)': p
                })

# Create a DataFrame from the results
df_resultsFA_new = pd.DataFrame(resultsFA_new)

# Optionally, format columns for easier reading
df_resultsFA_new["Electricity Price"] = df_resultsFA_new["Electricity Price C"].map(lambda x: f"{x:.2f}")
df_resultsFA_new["Formic Acid Price"] = df_resultsFA_new["Formic Acid Price C"].map(lambda x: f"{x:.0f}")
sensrow=df_resultsFA_new.shape[0]
print("Total Rows:", sensrow)

In [ ]:
min_valtot_ind=df_resultsFA_new['Profit ($)'].idxmin()
max_valtot_ind=df_resultsFA_new['Profit ($)'].idxmax()
min_prof_st=df_resultsFA_new.loc[min_valtot_ind, 'State']
max_prof_st=df_resultsFA_new.loc[max_valtot_ind, 'State']

print(f"Max Profit State: {max_prof_st}, Profit: ${df_resultsFA_new.loc[max_valtot_ind, 'Profit ($)']:,.0f}")
print(f"Min Profit State: {min_prof_st}, Profit: ${df_resultsFA_new.loc[min_valtot_ind, 'Profit ($)']:,.0f}")

In [ ]:
tol_tot = 100
zero_prof_rows_tot=df_resultsFA_new[(df_resultsFA_new['Profit ($)'] >= -tol_tot) &
                                    (df_resultsFA_new['Profit ($)'] <= tol_tot)]
state_rows=zero_prof_rows_tot.groupby('State').size()
print("Total Zero Profit Rows:", zero_prof_rows_tot.shape[0])
print("Total Zero Profit Rows in Each State:")
print(state_rows.to_string())

In [ ]:
nj_data=df_resultsFA_new[df_resultsFA_new['State']=="New Jersey"]

min_val=nj_data['Profit ($)'].min()
max_val=nj_data['Profit ($)'].max()

rows_count=nj_data.shape[0]
print("Total rows", rows_count)
# If you'd like to check where the profit is close to 0, you can filter here:
# Define a tolerance for finding profits close to 0 (can be positive or negative range)
tolerance = 100

# Filter rows where profit is between -tolerance and +tolerance
zero_profit_rows_nj = nj_data[(nj_data['Profit ($)'] >= -tolerance) &
                                (nj_data['Profit ($)'] <= tolerance)]

# Display the filtered results
print("Zero Profit:", zero_profit_rows_nj.shape[0])
# print(zero_profit_rows_nj[['State', 'CO2 Price', 'Formic Acid Price', 'Profit ($)']])
print(zero_profit_rows_nj)

## Interactive Heatmap of CO2 Price, Formic Acid Price and Plant Profit

In [ ]:
fig=px.imshow(pivot_FA, 
              labels={'x': 'Formic Acid Price', 'y': 'CO2 Price', 'color': 'Profit ($ Million)'},
              title='Heatmap of Profit by FA Price and CO2 Price', 
              color_continuous_scale='YlGnBu')
fig.show()